In [ ]:
!pip install tqdm pyswarm xgboost scikit-learn pandas numpy imbalanced-learn


  Preparing metadata (setup.py) ... done
  Created wheel for pyswarm: filename=pyswarm-0.6-py3-none-any.whl size=4464 sha256=a183e51f339d97b9460e5616f11a10114bbc167ea5493d018070262ff8eb4d27
  Stored in directory: /root/.cache/pip/wheels/bb/4f/ec/8970b83323e16aa95034da175454843947376614d6d5e9627f
Successfully built pyswarm


In [ ]:
import numpy as np
import pandas as pd
from tqdm import tqdm
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, roc_auc_score
from pyswarm import pso
from imblearn.over_sampling import SMOTE


In [ ]:

print("🔄 Loading Dataset...")
final_df = pd.read_csv("heart_failure_dataset.csv")


X = final_df.drop(columns=['subject_id', 'admittime', 'dischtime', 'dob', 'dod', 'readmission_30d'])
y = final_df['readmission_30d']

# One-Hot Encoding for categorical variables
X = pd.get_dummies(X, columns=['insurance', 'ethnicity', 'gender'], drop_first=True)

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Apply SMOTE for balancing classes
print("🔄 Applying SMOTE for Balancing Data...")
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Standardize features (for Logistic Regression)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_resampled)
X_test_scaled = scaler.transform(X_test)

print("✅ Data Preprocessing Completed!")


🔄 Loading Dataset...
🔄 Applying SMOTE for Balancing Data...
✅ Data Preprocessing Completed!


In [ ]:
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, roc_auc_score

# Initialize Models
xgb = XGBClassifier(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
rf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
lr = LogisticRegression(max_iter=500, random_state=42)

# Train Models
xgb.fit(X_train, y_train)
rf.fit(X_train, y_train)


# Predictions
y_pred_xgb = xgb.predict(X_test)
y_pred_rf = rf.predict(X_test)
# Evaluate Models
def evaluate_model(name, y_true, y_pred, model):
    accuracy = accuracy_score(y_true, y_pred)
    auc = roc_auc_score(y_true, model.predict_proba(X_test)[:, 1])
    print(f"{name} - Accuracy: {accuracy:.2f}, AUC: {auc:.2f}")

evaluate_model("XGBoost", y_test, y_pred_xgb, xgb)
evaluate_model("Random Forest", y_test, y_pred_rf, rf)



XGBoost - Accuracy: 0.87, AUC: 0.49
Random Forest - Accuracy: 0.87, AUC: 0.48


In [ ]:
from sklearn.model_selection import train_test_split

def pso_fitness(params, model_type):
    params = np.abs(params)  # Ensure parameters are positive

    if model_type == "xgb":
        model = XGBClassifier(n_estimators=int(params[0]), max_depth=int(params[1]), learning_rate=params[2], random_state=42)
    elif model_type == "rf":
        model = RandomForestClassifier(n_estimators=int(params[0]), max_depth=int(params[1]), random_state=42)
    else:  # Logistic Regression
        model = LogisticRegression(C=params[0], max_iter=500, solver='liblinear', random_state=42)

    # Train-Test Split inside PSO
    X_train_pso, X_val_pso, y_train_pso, y_val_pso = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

    model.fit(X_train_pso, y_train_pso)
    auc = roc_auc_score(y_val_pso, model.predict_proba(X_val_pso)[:, 1])

    return -auc  # Minimize negative AUC (maximize AUC)


In [ ]:
import os
import sys
import pickle

projectabspathname = os.path.abspath('projectname.pickle')
print(projectabspathname)
projectname = 'Training-model.ipynb'
projectpickle = open(str(projectabspathname),'wb')
pickle.dump(projectname, projectpickle)
projectpickle.close()


/content/projectname.pickle
